<a href="https://colab.research.google.com/github/ajw1587/Pytorch_Study/blob/main/12_Weight_Initialization_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch
!pip install torchvision

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
train_dataset = dsets.MNIST(root = '../data/',
                            train = True,
                            transform = transforms.ToTensor(),
                            download = True)
test_dataset = dsets.MNIST(root = '../data/',
                           train = False,
                           transform = transforms.ToTensor(),
                           download = True)

print(type(train_dataset))
print(train_dataset.data.shape)

print(type(test_dataset))
print(test_dataset.data.shape)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

<class 'torchvision.datasets.mnist.MNIST'>
torch.Size([60000, 28, 28])
<class 'torchvision.datasets.mnist.MNIST'>
torch.Size([10000, 28, 28])


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
data_loader = torch.utils.data.DataLoader(train_dataset,
                                          100,
                                          shuffle = True,
                                          drop_last = True)

In [6]:
# Model
class MNIST_Model(nn.Module):
  def __init__(self):
    super(MNIST_Model, self).__init__()
    self.linear1 = nn.Linear(28 * 28, 100)
    self.linear2 = nn.Linear(100, 200)
    self.linear3 = nn.Linear(200, 100)
    self.linear4 = nn.Linear(100, 10)

    torch.nn.init.xavier_uniform_(self.linear4.weight)

  def forward(self,x):
    x = self.linear1(x)
    x = F.relu(x)
    x = self.linear2(x)
    x = F.relu(x)
    x = F.relu(self.linear3(x))
    y = self.linear4(x)
    return y

In [7]:
model = MNIST_Model().to(device)

In [8]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [9]:
# training
total_batch = len(data_loader)
print(total_batch)

for epoch in range(5):
  avg_cost = 0

  for X, Y in data_loader:
    x = X.view(-1, 28 * 28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(x)

    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('[Epoch: {}] cost = {}'.format(epoch, avg_cost))
print('Learning Finished')

600
[Epoch: 0] cost = 0.26561257243156433
[Epoch: 1] cost = 0.1517496258020401
[Epoch: 2] cost = 0.12978333234786987
[Epoch: 3] cost = 0.11260710656642914
[Epoch: 4] cost = 0.10991360247135162
Learning Finished
